<a href="https://colab.research.google.com/github/lpoggetto/fiap_liveness/blob/main/trabalho_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# limpando a pasta
!rm -rf /content/*

In [2]:
 # instalando pacotes utilizados

!pip install git+https://github.com/hukkelas/DSFD-Pytorch-Inference.git
!pip install face_detection
!pip install mediapipe
!pip install dlib

print('pacotes instalados')

  Cloning https://github.com/hukkelas/DSFD-Pytorch-Inference.git to /tmp/pip-req-build-8uxnb8gb
  Running command git clone --filter=blob:none --quiet https://github.com/hukkelas/DSFD-Pytorch-Inference.git /tmp/pip-req-build-8uxnb8gb
  Resolved https://github.com/hukkelas/DSFD-Pytorch-Inference.git to commit dde9c7dd9cdc9254c2ca345222c86a8ecfa17f5b
  Preparing metadata (setup.py) ... done
pacotes instalados


### Importando os pacotes utilizados

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import bz2
import os
import face_detection
import mediapipe as mp
from scipy.spatial import distance as dist
import dlib
import warnings
warnings.filterwarnings('ignore')

#Exibição na mesma tela do Jupyter
%matplotlib inline
sns.set_style("whitegrid", {'axes.grid' : False})

# pacotes utilizados para tirar foto/videos
from IPython.display import display, Javascript, Image, HTML
from google.colab.output import eval_js
from base64 import b64decode
import threading

dlib.DLIB_USE_CUDA = True

In [4]:
# modelo pre-treinado dos landmarks usados no dlib
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
file_name = "shape_predictor_68_face_landmarks.dat.bz2"

# criando diretorio para salvar o modelo
os.makedirs('dlib_models', exist_ok=True)

# apontando para o diretorio
os.chdir('dlib_models')

# baixando o arquivo se já nao estiver baixado
if not os.path.exists(file_name):
  !wget {url}

# extraindo o arquivo.dat se ja nao estiver extraido
dat_file = "shape_predictor_68_face_landmarks.dat"
if not os.path.exists(dat_file):
  with bz2.BZ2File(file_name, "rb") as f_in, open(dat_file, "wb") as f_out:
    f_out.write(f_in.read())

# apagando o bz2 (compactado)
os.remove(file_name)

# voltando para o diretorio original
os.chdir('..')

print('modelo baixado')

--2025-01-03 14:51:04--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  78.4MB/s    in 0.8s    

2025-01-03 14:51:05 (78.4 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

modelo baixado


In [ ]:
# JavaScript code to capture video
html_code = """
<video id="video" width="640" height="480" autoplay></video>
<button id="startButton">Start Recording</button>
<button id="stopButton" disabled>Stop Recording</button>
<video id="playback" width="640" height="480" controls></video>
<a id="downloadLink" download="recorded-video.webm"></a>

<script>
    const video = document.querySelector('#video');
    const playback = document.querySelector('#playback');
    const downloadLink = document.querySelector('#downloadLink');
    const startButton = document.querySelector('#startButton');
    const stopButton = document.querySelector('#stopButton');

    navigator.mediaDevices.getUserMedia({ video: true }).then((stream) => {
        video.srcObject = stream;
        let mediaRecorder;
        const chunks = [];

        startButton.onclick = () => {
            mediaRecorder = new MediaRecorder(stream);
            mediaRecorder.start();
            startButton.disabled = true;
            stopButton.disabled = false;

            mediaRecorder.ondataavailable = (event) => chunks.push(event.data);
            mediaRecorder.onstop = () => {
                const blob = new Blob(chunks, { type: 'video/webm' });
                playback.src = URL.createObjectURL(blob);
                downloadLink.href = playback.src;
                downloadLink.textContent = 'Download Video';
                startButton.disabled = false;
                stopButton.disabled = true;
            };
        };

        stopButton.onclick = () => mediaRecorder.stop();
    });
</script>
"""

# Display the HTML interface
display(HTML(html_code))

In [6]:
# criando pasta para colocar o video gravado
os.makedirs('videos', exist_ok=True)

### Montando detector de faces

Utilizando o algoritmo DSFDDetector para detectar uma face

In [18]:
def calculo_distancia_olhos(eye):
    # Calcula a distância euclidiana entre os pontos dos olhos
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    # Calcula a razão de aspecto do olho
    ear = (A + B) / (2.0 * C)
    return ear

def detector_faces_video(video_path, output_path, threshhold_ear=0.3):
    # Criando o detector de faces
    detector = face_detection.build_detector(
        "DSFDDetector",
        confidence_threshold=0.5,
        nms_iou_threshold=0.3
    )
    # utilizando algoritmo de 68 pontos faciais
    preditor_marcos_faciais = dlib.shape_predictor(
        '/content/dlib_models/shape_predictor_68_face_landmarks.dat'
        )

    # abrindo o arquivo de video
    video = cv2.VideoCapture(video_path)

    fps = video.get(cv2.CAP_PROP_FPS)  # Get the original frame rate
    if fps == 0 or fps is None:  # Fallback if FPS is not retrieved correctly
      fps = 30  # Default to 30 FPS (you can adjust this if needed)


    # propriedades do video
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'VP80')  # WebM codec
    out = cv2.VideoWriter(output_path, fourcc, 15, (width, height))

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # convertendo para RGB
        rgb_frame = frame[:, :, ::-1]

        # detectando  face
        detections = detector.detect(rgb_frame)

        # desenhando o retangulo na face
        for det in detections:
            x1, y1, x2, y2, score = det
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Draw a red rectangle by default
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

            # utilizando dlib para detectar a face
            dlib_rect = dlib.rectangle(x1,y1,x2,y2)

            # marcos faciais
            landmarks = preditor_marcos_faciais(rgb_frame, dlib_rect)

            # coordenadas dos olhos
            left_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
            right_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]

            # calculando a distancia para ambos os olhos
            olho_esq = calculo_distancia_olhos(left_eye)
            olho_dir = calculo_distancia_olhos(right_eye)
            ear = (olho_esq + olho_dir) / 2

            # validacao de olhos piscando
            blink_detected = ear < threshhold_ear

            # Check if blink is detected
            if ear < threshhold_ear:
                # Update the rectangle to green if liveness is proven
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, 'Liveness OK', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # retorno do video
        out.write(frame)

    # Release resources
    video.release()
    out.release()
    print(f"Input video FPS: {fps}")
    print(f"Processed video saved to: {output_path}")

# Example usage
input_video_path = '/content/videos/recorded-video.webm'
output_video_path = '/content/videos/recorded-video_processado.webm'

# chamada da funcao criada anteriormente
detector_faces_video(input_video_path, output_video_path, threshhold_ear = 0.3)

Input video FPS: 62.5
Processed video saved to: /content/videos/recorded-video_processado.webm
